# Sample CloudSql Connection

# Need to set up for each instance - Add your VM's IP to the Authorized Networks
**1. Find the external IP of your JupyterLab VM**
https://console.cloud.google.com/compute/instances?project=adsp-34002-on02-sopho-scribe&authuser=1
 * Go to VM Instances
 * Find your JupyterLab VM
 * Copy the External IP address (looks like 34.91.100.45).

**2. Add the VM's IP to your Cloud SQL authorized networks**
https://console.cloud.google.com/sql/instances/currensee-sql/connections/networking?authuser=1&project=adsp-34002-on02-sopho-scribe
* Go to Cloud SQL instances. 
* Click your instance.
* Click Connections in the left sidebar.
* Scroll to Authorized networks → Add network.
* Name: anything like jupyterlab-vm
* Network: paste the external IP you just copied (e.g., 34.91.100.45/32)
* IMPORTANT: Add /32 to allow only that single IP.
* Click Save.

It will take ~30 seconds to update.

In [7]:
#%poetry add -q google-cloud-secret-manager==2.23.3
#%poetry add -q SQLAlchemy==2.0.40

In [8]:
#%poetry add psycopg2-binary sqlalchemy pandas

In [12]:
from google.cloud import secretmanager
import pandas as pd
from currensee.utils.db_utils import create_pg_engine
from sqlalchemy import text

## IMPORTANT
The cell below will only work if you have a .env file defined at `<fl>_currensee/currensee/.env` with the credentials 
defined in `<fl>_currensee/currensee/.env.example`.

Instructions are located within the `.env.example` file with how to fill out the credentials properly.

#### Create SQLAlchemy engine

In [2]:
# Define DB_NAME
DB_NAME = 'crm'

In [3]:
engine = create_pg_engine(
   db_name=DB_NAME
)

In [4]:
df_result = pd.read_sql("SELECT * FROM Employees limit 10", con=engine)
print(df_result)

                            employee_id first_name   last_name  \
0  8126969a-a976-47b5-9d1c-67a95de676e9       Jane  Moneypenny   
1  7786629e-c791-417a-b459-1b79e7db2b87     Amanda  Pennington   
2  749b00a0-5bae-4881-bebd-3526c086c453     Hayley      Garcia   
3  ef2ca01c-4c7b-4445-b840-8fbb658775bc      Dylan    Williams   
4  fb4485cd-6a4a-4985-860b-a52bf8cd2872       John      Montes   
5  43621c9d-0e36-4cfe-a84f-b089cc1f757d     Andrew      Guerra   
6  e48d378b-541a-4809-8e45-ac3d19158fed   Samantha    Reynolds   
7  a4ff2754-0908-4ba3-8f5d-40db55f9c4f9      James       Craig   
8  677fde0f-04fc-48fd-bcb3-c8668cc91959       Eric       Ortiz   
9  c9ceaed5-c321-40d9-88a6-e14415f722d8   Kristina      Glover   

                         title                           email  \
0         Relationship Manager   jane.moneypenny1@bankwell.com   
1  Senior Relationship Manager  amanda.pennington@bankwell.com   
2            Financial Advisor      hayley.garcia@bankwell.com   
3        

# Generate Fake Data

In [ ]:
# %poetry add faker

In [5]:
import pandas as pd
import random
import os
import re
from faker import Faker

# Initialize Faker instance
#Faker is a Python package that integrates fake data for you.

#Some hard coded data of publicly traded companies to be our mock clients


In [6]:

fake = Faker()

# Helper function to generate synthetic employee data
def generate_employee_data(num_employees=10, ourcompany_name = 'bankwell'):
    company = ourcompany_name
    employees = [
        {
            'employee_id': fake.unique.uuid4(),
            'first_name': "Jane",
            'last_name': "Moneypenny",
            'title': "Relationship Manager",
            'email': "jane.moneypenny1@bankwell.com",
            'phone': fake.phone_number(),
            'hire_date': fake.date_this_decade(),
            'department': 'Enterprise Investment',
            'market': 'San Fransisco'
        }
    ]

    for _ in range(num_employees-1):
        employee_id = fake.unique.uuid4()
        first_name = fake.first_name()
        last_name = fake.last_name()
        title = random.choice(['Finance Assistant', 'Financial Advisor', 'Senior Relationship Manager', 'Product Specialist', 'Relationship Manager'])
        phone = fake.phone_number()
        department = random.choice(['Enterprise Investment', 'Small Business Investment', 'Operations', 'Sales', 'Customer Support'])
        hire_date = fake.date_this_decade()
        market = random.choice(['San Fransisco', 'New York City', 'Boston', 'Denver', 'Los Angeles', 'Miami', 'Washington DC', 'Seattle', 'Dallas', 'Chicago'])
        company_clean = re.sub(r'\W+', '', company).lower()
        email = f"{first_name.lower()}.{last_name.lower()}@{company_clean}.com"


        employees.append({
            'employee_id': employee_id,
            'first_name': first_name,
            'last_name': last_name,
            'title': title,
            'email': email,
            'phone': phone,
            'hire_date': hire_date,
            'department': department,
            'market': market
        })
    return pd.DataFrame(employees)

# Helper function to generate point of contact and info for a Company
def generate_point_of_contact(company_name):
    account_id = fake.unique.uuid4()
    first_name = fake.first_name()
    last_name = fake.last_name()
    contact_title = random.choice(["Senior Director", "Manager", "Director", "VP", "Consultant"])
    phone = fake.phone_number()
    Website = fake.url()
    Location = random.choice(['San Fransisco', 'New York City', 'Boston', 'Denver', 'Los Angeles', 'Miami', 'Washington DC', 'Seattle', 'Dallas', 'Chicago'])
    AnnualRevenue = random.randint(1000000, 50000000)
    TotalAccountBal = random.randint(1000000, 50000000)

    # Clean Company name for use in email
    company_clean = re.sub(r'\W+', '', company_name).lower()
    email = f"{first_name.lower()}.{last_name.lower()}@{company_clean}.com"

    return {
        "account_id": account_id,
        "contact_first_name": first_name,
        "contact_last_name": last_name,
        "contact_title": contact_title,
        "phone": phone,
        "email": email,
        "website": Website,
        "location": Location,
        "annual_revenue": AnnualRevenue,
        "total_account_bal": TotalAccountBal
    }


def generate_account_data(num_accounts=5, clients_company = []):
# Build the data
    accounts = []
    for company in clients_company:
        contact = generate_point_of_contact(company["company"])
        record = {
          "company": company["company"],
          "industry": company["industry"],
          **contact
      }
        accounts.append(record)
    return pd.DataFrame(accounts)




# Helper function to generate synthetic opportunity data
def generate_opportunity_data(accounts_df, num_opportunities_per_account=3):
    opportunities = []
    for _, account in accounts_df.iterrows():
        num_opportunities = random.randint(1, num_opportunities_per_account)
        for _ in range(num_opportunities):
            opportunities.append({
                'opportunity_id': fake.unique.uuid4(),
                'account_id': account['account_id'],
                'opportunity_name': fake.bs(),
                'stage': random.choice(['Prospecting', 'Qualification', 'Proposal', 'Negotiation', 'Won', 'Lost', 'Closed']),
                'type': random.choice(['New Business', 'Existing Business', 'Renewal', 'Upsell']),
                'close_date': fake.date_this_year(),
                'amount': random.randint(50000, 500000),
            })
    return pd.DataFrame(opportunities)

# Helper function to generate synthetic employee-contact relationship data
def generate_employee_contact_data(employees_df, accounts_df, num_relationships_per_employee=2):
    relationships = []
    for _, employee in employees_df.iterrows():
        num_relationships = random.randint(1, num_relationships_per_employee)
        for _ in range(num_relationships):
            account = random.choice(accounts_df['account_id'].tolist())
            relationships.append({
                'employee_id': employee['employee_id'],
                'employee_first_name': employee['first_name'],
                'employee_last_name': employee['last_name'],
                'account_id': account,
                'company': accounts_df.loc[accounts_df['account_id'] == account, 'company'].iloc[0],
                'industry': accounts_df.loc[accounts_df['account_id'] == account, 'industry'].iloc[0],
                'contact_first_name': accounts_df.loc[accounts_df['account_id'] == account, 'contact_first_name'].iloc[0],
                'contact_last_name': accounts_df.loc[accounts_df['account_id'] == account, 'contact_last_name'].iloc[0],
                'contact_email': accounts_df.loc[accounts_df['account_id'] == account, 'email'].iloc[0],
                'contact_title': accounts_df.loc[accounts_df['account_id'] == account, 'contact_title'].iloc[0],
                'contact_phone': accounts_df.loc[accounts_df['account_id'] == account, 'phone'].iloc[0],
            })
    return pd.DataFrame(relationships)


def generate_portfolios(df_accounts, max_positions=10, instruments=[]):
    portfolio_records = []

    for _, row in df_accounts.iterrows():
        account_id = row["account_id"]
        company = row["company"]
        num_positions = random.randint(5, 10)
        positions = random.sample(instruments, num_positions)
        for symbol, instrument_type in positions:
            portfolio_records.append({
                "account_id": account_id,
                "company": company,
                "symbol": symbol,
                "instrument_type": instrument_type
            })

    return pd.DataFrame(portfolio_records)



In [7]:
# Right now companies can only be selected from this list of publicly traded companies 
# Likely want this to be random instead..
clients_company_info = [
        {"company": "Broadcom", "industry": "Technology"},
        {"company": "Cisco", "industry": "Technology"},
        {"company": "Palantir Technologies", "industry": "Technology"},
        {"company": "Fiserv", "industry": "Technology"},
        {"company": "Atlassian", "industry": "Technology"},
        {"company": "Leidos", "industry": "Technology"},
        {"company": "Duolingo", "industry": "Technology"},
        {"company": "Logitech", "industry": "Technology"},
        {"company": "Celestica", "industry": "Technology"},
        {"company": "Dropbox", "industry": "Technology"},
        {"company": "Plexus", "industry": "Technology"},
        {"company": "Silicon Laboratories", "industry": "Technology"},
        {"company": "Mobix Labs", "industry": "Technology"},
        {"company": "Mariott", "industry": "Hospitality"},
        {"company": "InterContinental Hotels Group", "industry": "Hospitality"},
        {"company": "Sonder Holdings", "industry": "Hospitality"},
        {"company": "Hyatt Hotels", "industry": "Hospitality"},
        {"company": "Royal Caribbean Cruises", "industry": "Hospitality"},
        {"company": "UnitedHealth", "industry": "Healthcare"},
        {"company": "Johnson & Johnson", "industry": "Healthcare"},
        {"company": "AbbVie", "industry": "Healthcare"},
        {"company": "Novo Nordisk", "industry": "Healthcare"},
        {"company": "Abbott Laboratories", "industry": "Healthcare"},
        {"company": "AstraZeneca", "industry": "Healthcare"},
        {"company": "Merck & Co", "industry": "Healthcare"},
        {"company": "Intuitive Surgical", "industry": "Healthcare"},
        {"company": "Medtronic", "industry": "Healthcare"},
        {"company": "Zoetis", "industry": "Healthcare"},
        {"company": "Humana", "industry": "Healthcare"},
        {"company": "Illumina", "industry": "Healthcare"},
        {"company": "Guardant Health", "industry": "Healthcare"},
        {"company": "Rhythm Pharmaceuticals", "industry": "Healthcare"},
        {"company": "Amedisys", "industry": "Healthcare"},
        {"company": "Rivian Automotive", "industry": "Automotive"},
        {"company": "Fordy", "industry": "Automotive"},
        {"company": "lululemon athletica", "industry": "Retail"},
        {"company": "DICK'S Sporting Goods", "industry": "Retail"},
        {"company": "GameStop Corp", "industry": "Retail"},
        {"company": "Texas Roadhouse", "industry": "Retail"},
        {"company": "Hasbro", "industry": "Retail"},
        {"company": "Mattel", "industry": "Retail"},
        {"company": "Wayfair", "industry": "Retail"},
        {"company": "Peloton", "industry": "Retail"},
        {"company": "Sally Beauty", "industry": "Retail"},
        {"company": "Lifetime Brand", "industry": "Retail"},
        {"company": "Allbirds", "industry": "Retail"},
        {"company": "Walmart", "industry": "Retail"},
        {"company": "Tyson Foods", "industry": "Retail"},
        {"company": "Sprouts Farmers Market", "industry": "Retail"},
        {"company": "Dollar Tree", "industry": "Retail"},
        {"company": "Stride", "industry": "Retail"},
        {"company": "Spectrum Brands", "industry": "Retail"},
        {"company": "Udemy", "industry": "Retail"},
        {"company": "Vital Farms", "industry": "Retail"},
        {"company": "Graham Holdings Company", "industry": "Retail"},
        {"company": "Hims & Hers Health", "industry": "Retail"},
        {"company": "Smithfield Foods", "industry": "Retail"},
        {"company": "Albertsons Companies", "industry": "Retail"},
        {"company": "Albany International", "industry": "Manufacturing"},
        {"company": "IT Tech Packaging", "industry": "Manufacturing"},
        {"company": "Lockheed Martin Corporation", "industry": "Manufacturing"},
        {"company": "Landstar System", "industry": "Manufacturing"},
        {"company": "Hexcel Corporation", "industry": "Manufacturing"},
        {"company": "AeroVironment", "industry": "Manufacturing"},
        {"company": "Matson", "industry": "Manufacturing"},
        {"company": "McGrath RentCorp", "industry": "Manufacturing"},
        {"company": "Mueller Industries", "industry": "Manufacturing"},
        {"company": "Dolby Laboratories", "industry": "Manufacturing"},
        {"company": "ManpowerGroup", "industry": "Manufacturing"},
        {"company": "Welltower", "industry": "RealEstate"},
        {"company": "Iron Mountain Incorporated", "industry": "RealEstate"},
        {"company": "Camden Property", "industry": "RealEstate"},
        {"company": "CubeSmart", "industry": "RealEstate"},
        {"company": "Federal Realty Investment Trust", "industry": "RealEstate"},
        {"company": "Essential Properties Realty", "industry": "RealEstate"},
        {"company": "Compass", "industry": "RealEstate"},
        {"company": "Medical Properties Trust", "industry": "RealEstate"},
        {"company": "Broadstone", "industry": "RealEstate"},
        {"company": "Ladder Capital Corp", "industry": "RealEstate"},
        {"company": "Peakstone Realty Trus", "industry": "RealEstate"},
        {"company": "Fathom Holdings", "industry": "RealEstate"},
        {"company": "Presidio Property Trust", "industry": "RealEstate"},
        {"company": "Service Properties Trust", "industry": "RealEstate"},
    ]

stocks = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'TSLA', 'NVDA', 'META', 'JPM', 'V', 'UNH']
bonds = ['US10Y', 'US30Y', 'CORP1', 'CORP2', 'MUNI1', 'MUNI2']
mutual_funds = ['VFIAX', 'SWPPX', 'FXAIX', 'VTSAX', 'FZROX', 'SPY']
# All symbols with types
instruments = (
    [(sym, 'Stock') for sym in stocks] +
    [(sym, 'Bond') for sym in bonds] +
    [(sym, 'Mutual Fund') for sym in mutual_funds]
)

# Generate synthetic data for the tables
employees_df = generate_employee_data(num_employees=100, ourcompany_name = 'bankwell')
accounts_df = generate_account_data(num_accounts=1000, clients_company = clients_company_info)
opportunities_df = generate_opportunity_data(accounts_df, num_opportunities_per_account=3)
contacts_df = generate_employee_contact_data(employees_df, accounts_df, num_relationships_per_employee=30)
portfolio_df = generate_portfolios(accounts_df, max_positions=10, instruments = instruments)

# Print the first few rows of each DataFrame
print("Employees Data:")
print(employees_df.head())

print("\nAccounts Data:")
print(accounts_df.head())

print("\nOpportunities Data:")
print(opportunities_df.head())

print("\nEmployee-Contact Relationships Data:")
print(contacts_df.head())

print("\nPortfolio Info for Each Account")
print(portfolio_df.head())





Employees Data:
                            employee_id first_name   last_name  \
0  d94e0bbc-4e22-4b90-b4cc-6dc980516d86       Jane  Moneypenny   
1  0e357992-e05f-47c5-aa41-0ca5ce181747    Michael      Walker   
2  dfbe5f2a-cb67-4912-93d7-3ec1e1673c19   Jennifer       Duran   
3  a05efef7-78ae-4557-931a-4edcad1e1c66      Tyler      Mullen   
4  b47e6331-fd75-41e5-bd46-09f430a92c2c       John    Crawford   

                         title                          email  \
0         Relationship Manager  jane.moneypenny1@bankwell.com   
1  Senior Relationship Manager    michael.walker@bankwell.com   
2  Senior Relationship Manager    jennifer.duran@bankwell.com   
3           Product Specialist      tyler.mullen@bankwell.com   
4            Financial Advisor     john.crawford@bankwell.com   

                phone   hire_date                 department         market  
0     +1-745-839-3339  2022-10-03      Enterprise Investment  San Fransisco  
1  277-833-2788x23496  2024-01-07       

# Load to database

### Employees Table

In [8]:
employees_df.head()

,employee_id,first_name,last_name,title,email,phone,hire_date,department,market
0,d94e0bbc-4e22-4b90-b4cc-6dc980516d86,Jane,Moneypenny,Relationship Manager,jane.moneypenny1@bankwell.com,+1-745-839-3339,2022-10-03,Enterprise Investment,San Fransisco
1,0e357992-e05f-47c5-aa41-0ca5ce181747,Michael,Walker,Senior Relationship Manager,michael.walker@bankwell.com,277-833-2788x23496,2024-01-07,Customer Support,Dallas
2,dfbe5f2a-cb67-4912-93d7-3ec1e1673c19,Jennifer,Duran,Senior Relationship Manager,jennifer.duran@bankwell.com,393.333.2384x779,2021-08-18,Small Business Investment,Dallas
3,a05efef7-78ae-4557-931a-4edcad1e1c66,Tyler,Mullen,Product Specialist,tyler.mullen@bankwell.com,001-250-318-4682,2021-03-31,Operations,Miami
4,b47e6331-fd75-41e5-bd46-09f430a92c2c,John,Crawford,Financial Advisor,john.crawford@bankwell.com,836-657-9713,2024-08-03,Small Business Investment,Dallas


In [9]:
employees_df.shape

(100, 9)

In [10]:
employees_df.to_sql('employees', engine, if_exists='replace', index=False)

100

In [13]:
alter_sql = """
ALTER TABLE "employees"
ADD PRIMARY KEY ("employee_id");
"""

with engine.connect() as connection:
    connection.execute(text(alter_sql))

In [14]:
df = pd.read_sql("SELECT * from employees", con=engine)
df.head()

,employee_id,first_name,last_name,title,email,phone,hire_date,department,market
0,d94e0bbc-4e22-4b90-b4cc-6dc980516d86,Jane,Moneypenny,Relationship Manager,jane.moneypenny1@bankwell.com,+1-745-839-3339,2022-10-03,Enterprise Investment,San Fransisco
1,0e357992-e05f-47c5-aa41-0ca5ce181747,Michael,Walker,Senior Relationship Manager,michael.walker@bankwell.com,277-833-2788x23496,2024-01-07,Customer Support,Dallas
2,dfbe5f2a-cb67-4912-93d7-3ec1e1673c19,Jennifer,Duran,Senior Relationship Manager,jennifer.duran@bankwell.com,393.333.2384x779,2021-08-18,Small Business Investment,Dallas
3,a05efef7-78ae-4557-931a-4edcad1e1c66,Tyler,Mullen,Product Specialist,tyler.mullen@bankwell.com,001-250-318-4682,2021-03-31,Operations,Miami
4,b47e6331-fd75-41e5-bd46-09f430a92c2c,John,Crawford,Financial Advisor,john.crawford@bankwell.com,836-657-9713,2024-08-03,Small Business Investment,Dallas


### Clients Contact table

In [15]:
accounts_df.head()

,company,industry,account_id,contact_first_name,contact_last_name,contact_title,phone,email,website,location,annual_revenue,total_account_bal
0,Broadcom,Technology,dff375b6-f4b7-4635-924f-c755f010e308,Candace,Wagner,Senior Director,350-421-9160x759,candace.wagner@broadcom.com,https://ruiz.com/,Miami,5590198,49545235
1,Cisco,Technology,f9768e8b-2f4f-4802-8e28-3d0beb61d64a,Melanie,Valenzuela,Senior Director,268-462-1934x7197,melanie.valenzuela@cisco.com,http://boyd.com/,Boston,34797528,19938030
2,Palantir Technologies,Technology,dd2f8718-b1fe-43dd-8e6a-df0ec6d91331,Katherine,Jones,Manager,366.241.7112,katherine.jones@palantirtechnologies.com,http://www.bailey.com/,Los Angeles,17960396,30986263
3,Fiserv,Technology,a34b0aa4-eb1e-4121-8d23-957a189d3ec5,Tiffany,Gibson,VP,001-375-879-9386,tiffany.gibson@fiserv.com,https://miller.com/,Seattle,40152371,18725120
4,Atlassian,Technology,032cb0a4-04b1-4697-ad73-98b4082537eb,Lauren,Stewart,VP,365.770.3181x2552,lauren.stewart@atlassian.com,http://www.silva-mitchell.com/,Los Angeles,32782153,12578391


In [16]:
accounts_df.shape

(83, 12)

In [17]:
accounts_df.to_sql('clients_contact', engine, if_exists='replace', index=False)

83

In [18]:
alter_sql = """
ALTER TABLE "clients_contact"
ADD PRIMARY KEY ("account_id");
"""

with engine.connect() as connection:
    connection.execute(text(alter_sql))

### Client alignment table

In [19]:
contacts_df.head()

,employee_id,employee_first_name,employee_last_name,account_id,company,industry,contact_first_name,contact_last_name,contact_email,contact_title,contact_phone
0,d94e0bbc-4e22-4b90-b4cc-6dc980516d86,Jane,Moneypenny,8ff6bfed-1434-4b5c-9b56-1a2f59afe07b,CubeSmart,RealEstate,Charles,Bruce,charles.bruce@cubesmart.com,Senior Director,9065927477
1,d94e0bbc-4e22-4b90-b4cc-6dc980516d86,Jane,Moneypenny,e48d05b0-a6db-4207-9783-93dbff879693,Broadstone,RealEstate,Ann,Dennis,ann.dennis@broadstone.com,VP,891-933-0605x0515
2,d94e0bbc-4e22-4b90-b4cc-6dc980516d86,Jane,Moneypenny,aef6c212-ca7b-4520-8250-4ff0c090da4a,Landstar System,Manufacturing,Michelle,Smith,michelle.smith@landstarsystem.com,Consultant,526-549-3474x255
3,d94e0bbc-4e22-4b90-b4cc-6dc980516d86,Jane,Moneypenny,5d37ddf9-b3a0-4e47-916e-24ea0550c06e,IT Tech Packaging,Manufacturing,Mary,Obrien,mary.obrien@ittechpackaging.com,Senior Director,6374165818
4,d94e0bbc-4e22-4b90-b4cc-6dc980516d86,Jane,Moneypenny,a8eca87d-73eb-4172-a081-01194b319b96,Camden Property,RealEstate,Cody,Thompson,cody.thompson@camdenproperty.com,Senior Director,248.403.3815


In [23]:
client_alignment = contacts_df.drop_duplicates()

In [24]:
client_alignment.to_sql('client_alignment', engine, if_exists='replace', index=False)

326

In [25]:
alter_sql = """
ALTER TABLE "client_alignment"
ADD PRIMARY KEY ("account_id", "employee_id");
"""

with engine.connect() as connection:
    connection.execute(text(alter_sql))

### Portfolio Table

In [26]:
portfolio_df.head()

,account_id,company,symbol,instrument_type
0,dff375b6-f4b7-4635-924f-c755f010e308,Broadcom,US30Y,Bond
1,dff375b6-f4b7-4635-924f-c755f010e308,Broadcom,MUNI1,Bond
2,dff375b6-f4b7-4635-924f-c755f010e308,Broadcom,AMZN,Stock
3,dff375b6-f4b7-4635-924f-c755f010e308,Broadcom,MUNI2,Bond
4,dff375b6-f4b7-4635-924f-c755f010e308,Broadcom,MSFT,Stock


In [27]:
portfolio_df.to_sql('portfolio', engine, if_exists='replace', index=False)

635

In [28]:
alter_sql = """
ALTER TABLE "portfolio"
ADD PRIMARY KEY ("account_id", "symbol");
"""

with engine.connect() as connection:
    connection.execute(text(alter_sql))

# Test Results

In [29]:
pd.read_sql("SELECT * FROM portfolio limit 10", con=engine)

,account_id,company,symbol,instrument_type
0,dff375b6-f4b7-4635-924f-c755f010e308,Broadcom,US30Y,Bond
1,dff375b6-f4b7-4635-924f-c755f010e308,Broadcom,MUNI1,Bond
2,dff375b6-f4b7-4635-924f-c755f010e308,Broadcom,AMZN,Stock
3,dff375b6-f4b7-4635-924f-c755f010e308,Broadcom,MUNI2,Bond
4,dff375b6-f4b7-4635-924f-c755f010e308,Broadcom,MSFT,Stock
5,dff375b6-f4b7-4635-924f-c755f010e308,Broadcom,CORP1,Bond
6,dff375b6-f4b7-4635-924f-c755f010e308,Broadcom,VTSAX,Mutual Fund
7,dff375b6-f4b7-4635-924f-c755f010e308,Broadcom,FZROX,Mutual Fund
8,dff375b6-f4b7-4635-924f-c755f010e308,Broadcom,UNH,Stock
9,f9768e8b-2f4f-4802-8e28-3d0beb61d64a,Cisco,US10Y,Bond


In [30]:
pd.read_sql("SELECT * FROM client_alignment limit 10", con=engine)

,employee_id,employee_first_name,employee_last_name,account_id,company,industry,contact_first_name,contact_last_name,contact_email,contact_title,contact_phone
0,d94e0bbc-4e22-4b90-b4cc-6dc980516d86,Jane,Moneypenny,8ff6bfed-1434-4b5c-9b56-1a2f59afe07b,CubeSmart,RealEstate,Charles,Bruce,charles.bruce@cubesmart.com,Senior Director,9065927477
1,d94e0bbc-4e22-4b90-b4cc-6dc980516d86,Jane,Moneypenny,e48d05b0-a6db-4207-9783-93dbff879693,Broadstone,RealEstate,Ann,Dennis,ann.dennis@broadstone.com,VP,891-933-0605x0515
2,d94e0bbc-4e22-4b90-b4cc-6dc980516d86,Jane,Moneypenny,aef6c212-ca7b-4520-8250-4ff0c090da4a,Landstar System,Manufacturing,Michelle,Smith,michelle.smith@landstarsystem.com,Consultant,526-549-3474x255
3,d94e0bbc-4e22-4b90-b4cc-6dc980516d86,Jane,Moneypenny,5d37ddf9-b3a0-4e47-916e-24ea0550c06e,IT Tech Packaging,Manufacturing,Mary,Obrien,mary.obrien@ittechpackaging.com,Senior Director,6374165818
4,d94e0bbc-4e22-4b90-b4cc-6dc980516d86,Jane,Moneypenny,a8eca87d-73eb-4172-a081-01194b319b96,Camden Property,RealEstate,Cody,Thompson,cody.thompson@camdenproperty.com,Senior Director,248.403.3815
5,d94e0bbc-4e22-4b90-b4cc-6dc980516d86,Jane,Moneypenny,4d496d3c-3ba5-433d-a38e-414cbbb68691,Zoetis,Healthcare,Caitlin,Mcdowell,caitlin.mcdowell@zoetis.com,VP,562.380.4088x4476
6,d94e0bbc-4e22-4b90-b4cc-6dc980516d86,Jane,Moneypenny,506df24b-a3be-4980-9dea-14842393545d,Mobix Labs,Technology,Michael,Murphy,michael.murphy@mobixlabs.com,Senior Director,896-988-8770x48472
7,d94e0bbc-4e22-4b90-b4cc-6dc980516d86,Jane,Moneypenny,1eff44a9-f2d2-4cfa-a5b0-4ab351be5462,Lifetime Brand,Retail,Becky,Beard,becky.beard@lifetimebrand.com,VP,(263)562-1468x155
8,d94e0bbc-4e22-4b90-b4cc-6dc980516d86,Jane,Moneypenny,a34b0aa4-eb1e-4121-8d23-957a189d3ec5,Fiserv,Technology,Tiffany,Gibson,tiffany.gibson@fiserv.com,VP,001-375-879-9386
9,d94e0bbc-4e22-4b90-b4cc-6dc980516d86,Jane,Moneypenny,f56e1420-dde4-4414-8fb1-03cfda615f27,Logitech,Technology,Devon,Suarez,devon.suarez@logitech.com,Consultant,001-774-380-9228x09473


In [31]:
import datetime
import pytz

print(f"Notebook last execution time: {datetime.datetime.now(pytz.timezone('US/Central')).strftime('%a, %d %B %Y %H:%M:%S')}")

Notebook last execution time: Sun, 27 April 2025 22:12:10
